<a href="https://colab.research.google.com/github/amanichivilkar/Books-Recommendation-System/blob/main/Amani_Chivilkar_Content_Based_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

During the last few decades, with the rise of Youtube, Amazon, Netflix, and many other such
web services, recommender systems have taken more and more place in our lives. From
e-commerce (suggest to buyers articles that could interest them) to online advertisement
(suggest to users the right contents, matching their preferences), recommender systems are
today unavoidable in our daily online journeys.
In a very general way, recommender systems are algorithms aimed at suggesting relevant
items to users (items being movies to watch, text to read, products to buy, or anything else
depending on industries).
Recommender systems are really critical in some industries as they can generate a huge
amount of income when they are efficient or also be a way to stand out significantly from
competitors. The main objective is to create a book recommendation system for users.

 **Content**
--------------
The Book-Crossing dataset comprises 3 files.
*  Users
---
Contains the users. Note that user IDs (User-ID) have been anonymized and map to
integers. Demographic data is provided (Location, Age) if available. Otherwise, these
fields contain NULL values. 
*  Books
---------
Books are identified by their respective ISBN. Invalid ISBNs have already been removed
from the dataset. Moreover, some content-based information is given (Book-Title,
Book-Author, Year-Of-Publication, Publisher), obtained from Amazon Web
Services. Note that in the case of several authors, only the first is provided. URLs linking
to cover images are also given, appearing in three different flavors (Image-URL-S,
Image-URL-M, Image-URL-L), i.e., small, medium, large. These URLs point to the Amazon website.
*   Ratings
------------
Contains the book rating information. Ratings (Book-Rating) are either explicit,
expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit,
expressed by 0.

## **Import data and libraries**

In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
a = np.random.randn(12288, 150)
a.shape

(12288, 150)

In [ ]:
b = np.random.randn(150, 45) 

In [ ]:
c=np.dot(a,b)
c.shape

(12288, 45)

In [ ]:
books = pd.read_csv('/content/drive/MyDrive/data/data_book_recommendation/Books.csv')
ratings = pd.read_csv('/content/drive/MyDrive/data/data_book_recommendation/Ratings.csv')
user = pd.read_csv('/content/drive/MyDrive/data/data_book_recommendation/Users.csv')

In [ ]:
books.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], inplace=True)
books.rename(columns={'User-ID':'user_id','Book-Title':'title', 'Book-Rating':'rating','Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher',
                    'Book-Rating':'rating'}, inplace=True)
print(len(books))
books.head(2)

271360


,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


In [ ]:
# Working on books dataframe
books[(books['title']=='Selected Poems')].head()

,ISBN,title,author,year,publisher
4523,081120958X,Selected Poems,William Carlos Williams,1985,New Directions Publishing Corporation
39416,0811201465,Selected Poems,K. Patchen,1957,New Directions Publishing Corporation
41316,0679750800,Selected Poems,Rita Dove,1993,Vintage Books USA
106885,0060931744,Selected Poems,Gwendolyn Brooks,1999,Perennial
118775,0517101548,Selected Poems,John Donne,1994,Gramercy Books


*  Since same title has different author, in order to differentiate between the title of same nae we combine the tile with the auther 

*  And since the same book has different ISBN we cant use it insted we will combine the title and auther , and create a title_id for each unique title

In [ ]:
# combining tiltle with author to differentiate bet the books with same title
books['title']=books['title'] + " " + books['author']

In [ ]:
# Generating book_id for each unique book title
books['book_id'] = books[['title']].sum(axis=1).map(hash)

In [ ]:
books.head()

,ISBN,title,author,year,publisher,book_id
0,0195153448,Classical Mythology Mark P. O. Morford,Mark P. O. Morford,2002,Oxford University Press,-7293735737548367065
1,0002005018,Clara Callan Richard Bruce Wright,Richard Bruce Wright,2001,HarperFlamingo Canada,7149617095307170465
2,0060973129,Decision in Normandy Carlo D'Este,Carlo D'Este,1991,HarperPerennial,4608496659329701630
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,1160329172564954229
4,0393045218,The Mummies of Urumchi E. J. W. Barber,E. J. W. Barber,1999,W. W. Norton &amp; Company,-8279562061523982204


In [ ]:
print(len(ratings))
ratings.rename(columns={'User-ID':'user_id', 'Book-Rating':'rating'}, inplace=True)
ratings.head(2)

1149780


,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


## **Data Preprocessing**

In [ ]:
# We select those user who has given >= 50 ratings 
df1=ratings.groupby(['user_id'])['rating'].count().reset_index()
list_of_imp_user=list(df1[df1['rating']>50]['user_id'])
len(list_of_imp_user)

3371

In [ ]:
# Get rating dataframe of the user who has given >= 50 ratings
ratings=ratings[ratings['user_id'].isin(list_of_imp_user)]
print(len(ratings))
ratings.head()

765672


,user_id,ISBN,rating
173,276847,0446364193,0
174,276847,3257200552,5
175,276847,3379015180,0
176,276847,3404145909,8
177,276847,3404148576,8


In [ ]:
# merge ratings with books
df=ratings.merge(books, on='ISBN')
print(len(df))
df.head()

700848


,user_id,ISBN,rating,title,author,year,publisher,book_id
0,276847,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760
1,278418,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760
2,5483,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760
3,7346,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760
4,8362,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760


In [ ]:
print(f"unique title = {df['title'].nunique()}")
print(f"unique ISBN = {df['ISBN'].nunique()}")

unique title = 206071
unique ISBN = 221678


## **Final DataFrame**

In [ ]:
# The since a user reads only one book of one kind.. we will remove a row in which a user has read 2 book of same kind
df.drop_duplicates(['user_id','title'], inplace=True)
print(len(df))
df.head()

697534


,user_id,ISBN,rating,title,author,year,publisher,book_id
0,276847,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760
1,278418,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760
2,5483,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760
3,7346,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760
4,8362,0446364193,0,Along Came a Spider (Alex Cross Novels) James ...,James Patterson,1993,Warner Books,6935983676206818760


In [ ]:
df.dropna(inplace=True)

## **Train Test Split**

*  **Befor splitting df we need to remove user_id with one value count or else stratify on item_id in train_test_split will show error**

In [ ]:
x=df[['user_id']].value_counts().reset_index()
value1=list(x[x[0]==1]['user_id'])

In [ ]:
df=df[~df['user_id'].isin(value1)]
df['user_id'].value_counts().tail(2)

152464    3
76440     2
Name: user_id, dtype: int64

In [ ]:
# TrainTest split df
train_df, test_df = train_test_split(df, stratify=df['user_id'] ,test_size=0.20, random_state=42)

In [ ]:
# Indexing the User-ID to speed up the searches during evaluation
df = df.set_index('user_id')
train_df = train_df.set_index('user_id')
test_df = test_df.set_index('user_id')

## **You need to study How TF-ID vectorizer woks in order to get better accuracy**

*  ISBN=014028009X title=Bridget Jones's Diary	year=1999	
*  ISBN=0330375253 title=Bridget Jones's Diary	year=2001

## **Vectorize the title**

In [ ]:
# Get all the stop words
stopword_list=stopwords.words('english') 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1, 2) , min_df=0.003, max_df=0.5,
                     max_features=5000 , stop_words=stopword_list)

In [ ]:
#  Drop nan values befor vectorizing title and author
books.dropna(inplace=True)

In [ ]:
vectorized_matrix=vectorizer.fit_transform(books['title'])
vectorized_matrix_feature_name=vectorizer.get_feature_names()

In [ ]:
# Put all the books title in a list
books_ids=books['book_id'].tolist()
books_ids

In [ ]:
def get_item_profile(item_id):
    idx = books_ids.index(item_id)
    item_profile = vectorized_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['book_id'])
    
    user_item_strengths = np.array(interactions_person_df['rating']).reshape(-1,1)
    
    # Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) /np.sum(user_item_strengths)
    #user_profile_norm = normalize(user_item_strengths_weighted_avg)
    return user_item_strengths_weighted_avg

def build_users_profiles(): 
    interactions_indexed_df = df[df['book_id'].isin(books['book_id'])]
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [ ]:
book_id= -7293735737548367065

In [ ]:
for i in [book_id]:
      idx = books_ids.index(i)
      item_profile = vectorized_matrix[idx:idx+1]
      print(item_profile)

  (0, 136)	1.0


In [ ]:
item_profiles_list = [get_item_profile(x) for x in books_ids[0:5]]
item_profiles = scipy.sparse.vstack(item_profiles_list)
print(item_profiles) 

  (0, 136)	1.0
  (1, 170)	1.0
  (3, 82)	0.7417688707518302
  (3, 191)	0.6706556063909066


In [ ]:
user_item_strengths=np.array(df['rating'][0:5]).reshape(-1,1)
user_item_strengths

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [ ]:
np.sum(user_item_strengths)

0

In [ ]:
user_profiles = build_users_profiles()
len(user_profiles)

3369

*  user_profiles looks like a dictionary with keys as user and values as user profile
-------
{ -9223121837663643404 : array( [ [0.00679228, 0.01231635, 0.        , ..., 0.        , 0.        ,
         0.        ] ] ) , 
 -9212075797126931087  : array( [ [0.        , 0.02568444, 0.        , ..., 0.        , 0.00905023,
         0.        ] ] ) }

### Let's take a look at a particular user profile. It is a unit vector of 5000 length. The value in each position represents how relevant is a token (unigram or bigram) for the selected user

In [ ]:
user_id=276847

In [ ]:
# zip feature_name and user_profile
a=zip(vectorizer.get_feature_names() , user_profiles[user_id].flatten().tolist()[0])

In [ ]:
# Create a DataFrame of the matrix feature name and its 
user_df=pd.DataFrame( sorted(a), columns=['token', 'relevance']).sort_values("relevance",ascending=False)

In [ ]:
user_df.head(10)

,token,relevance
56,der,0.248740
60,die,0.185939
77,george,0.128292
70,elizabeth,0.125502
206,und,0.101812
89,harry,0.075526
57,des,0.065645
209,von,0.038183
173,roman,0.036535
49,das,0.035976


### Looking at this user's profile, it appears that the top relevant tokens really represent  professional interests in **machine learning**, **deep learning**, **artificial intelligence** and **google cloud platform**!